### 📘 Lesson 5: Market Model

<div style="display: flex; align-items: center; justify-content: space-between;">
  <div>
    <h3>Course presenters</h3>
    <ul>
      <li><strong>Priyesh Gosai</strong> - Energy Systems Modeler and Training Coordinator</li>
      <li><strong>Dr. Fabian Hofmann</strong> - Senior Optimization and Energy System Modelling Expert</li>
    </ul>
  </div>
  <div>
    <a href="https://openenergytransition.org/index.html">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="OET">
    </a>
  </div>
</div>


#### 🎯 Learning Objectives  



* Run a demonstration model using real data. 
* Solve the Day ahead and intra day model. 
* Analyse the results.

---

#### 📈 Market Model

In [17]:
import pypsa
import pandas as pd
import numpy as np
from training_scripts import *

In [18]:
input_file_name = 'data/Lesson5_model.xlsx'
path = convert_selected_sheets_to_csv(input_file_name, 'lesson5_csv_folder')

INFO:root:Converted snapshots to CSV.
INFO:root:Converted buses to CSV.
INFO:root:Converted carriers to CSV.
INFO:root:Converted generators to CSV.
INFO:root:Converted generators-p_max_pu to CSV.
INFO:root:Converted links to CSV.
INFO:root:Converted loads to CSV.
INFO:root:Converted loads-p_set to CSV.
INFO:root:Converted storage_units to CSV.
INFO:root:Converted storage_units-inflow to CSV.
INFO:root:Conversion complete. CSV files are saved in 'lesson5_csv_folder'
INFO:root:Excel file closed successfully.


In [19]:
network = pypsa.Network(path)

INFO:pypsa.io:Imported network lesson5_csv_folder has buses, carriers, generators, links, loads, storage_units


In [20]:
dispatchable_generators = network.generators[network.generators.carrier.isin(['coal', 'biomass', 'gas'])].index.tolist()

network.generators.loc[network.generators["carrier"] == "coal", "p_min_pu"] = 0.1
network.generators.loc[network.generators["carrier"] == "biomass", "p_min_pu"] = 0.1

network.generators.loc[network.generators["carrier"] == "coal", "ramp_limit_up"] = 0.25
network.generators.loc[network.generators["carrier"] == "biomass", "ramp_limit_up"] = 0.25

network.generators.loc[network.generators["carrier"] == "coal", "ramp_limit_down"] = 0.25
network.generators.loc[network.generators["carrier"] == "biomass", "ramp_limit_down"] = 0.25

network.generators.loc[network.generators["carrier"] == "coal", "min_up_time"] = 4
network.generators.loc[network.generators["carrier"] == "biomass", "min_up_time"] = 4

network.generators.loc[network.generators["carrier"] == "coal", "min_down_time"] = 10
network.generators.loc[network.generators["carrier"] == "biomass", "min_down_time"] = 10


💧 **Setting Water Availability for Hydropower Plants**


⚡ **Modeling Demand Side Management (DSM)**  





In [21]:
network.optimize(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 6/6 [00:00<00:00, 48.18it/s]
INFO:linopy.io: Writing time: 1.35s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 52800 primals, 199136 duals
Objective: 3.09e+09
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-fix-p-ramp_limit_up, Generator-fix-p-ramp_limit_down, Link-fix-p-lower, Link-fix-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


('ok', 'optimal')

In [22]:
del network.model

network_id = network.copy()


In [23]:
# Compute daily sum of generator output
daily_sum = network_id.generators_t.p.groupby(network_id.generators_t.p.index.date).sum()

daily_sum= daily_sum.loc[:, daily_sum.columns.isin(dispatchable_generators)]

In [24]:
# Loop through each timestep
for timestamp in network_id.snapshots:
    current_day = timestamp.date()
    
    # Get generators with zero output on the current day
    zero_output_generators = daily_sum.loc[current_day][daily_sum.loc[current_day] == 0].index
    
    # Set p_max_pu and p_min_pu to zero for the current timestamp
    network_id.generators_t.p_max_pu.loc[timestamp, zero_output_generators] = 0
    network_id.generators_t.p_min_pu.loc[timestamp, zero_output_generators] = 0


# Ensure selected generators are committable
network_id.generators.loc[dispatchable_generators, "committable"] = True

In [25]:
input_file_name

'data/Lesson5_model.xlsx'

In [26]:
# Update demand with actual demand

actual_demand_df = pd.read_excel(input_file_name,sheet_name='actual-demand', index_col=0, parse_dates=True)
network_id_snapshots = network_id.snapshots  # Get existing snapshots in the network
actual_demand_df = actual_demand_df.loc[actual_demand_df.index.intersection(network_id_snapshots)]

for col in actual_demand_df.columns:
    if col in network_id.loads_t.p_set.columns:
        network_id.loads_t.p_set[col] = actual_demand_df[col]

In [27]:
# Adjust renewable profiles with some noise 
# Define noise level
noise_level = 0.01  

# Generate random noise
noise = np.random.uniform(-noise_level, noise_level, size=network_id.generators_t.p_max_pu.shape)

# Apply noise to p_max_pu values
network_id.generators_t.p_max_pu *= (1 + noise)

In [28]:
network_id.optimize()

DatetimeIndex(['2015-01-05 00:00:00', '2015-01-05 01:00:00',
               '2015-01-05 02:00:00', '2015-01-05 03:00:00',
               '2015-01-05 04:00:00', '2015-01-05 05:00:00',
               '2015-01-05 06:00:00', '2015-01-05 07:00:00',
               '2015-01-05 08:00:00', '2015-01-05 09:00:00',
               '2015-01-05 10:00:00', '2015-01-05 11:00:00',
               '2015-01-05 12:00:00', '2015-01-05 13:00:00',
               '2015-01-05 14:00:00', '2015-01-05 15:00:00',
               '2015-01-05 16:00:00', '2015-01-05 17:00:00',
               '2015-01-05 18:00:00', '2015-01-05 19:00:00',
               '2015-01-05 20:00:00', '2015-01-05 21:00:00',
               '2015-01-05 22:00:00', '2015-01-05 23:00:00'],
              dtype='datetime64[ns]', name='snapshot', freq=None)
DatetimeIndex(['2015-01-05 00:00:00', '2015-01-05 01:00:00',
               '2015-01-05 02:00:00', '2015-01-05 03:00:00',
               '2015-01-05 04:00:00', '2015-01-05 05:00:00',
               '20

('ok', 'optimal')

```


```

In [29]:
network.generators_t.p[dispatchable_generators].plot()

c:\Users\gosai\anaconda3\envs\flex_training_env\lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [30]:
network.buses_t.marginal_price.plot()

c:\Users\gosai\anaconda3\envs\flex_training_env\lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



###
---